# 📊 Stock forecast & trading strategy with Nixtla & yFinance

This notebook shows how to analyze historical stock market data using yFinance and the Nixtla framework to derive a simple trading strategy. 

We use:
- `yFinance` to retrieve financial data
- `NeuralForecast` (RNN) for price prediction
- Fundamental data for decision support

# Setup & Imports

In [1]:
import sys
import os
sys.path.append(os.path.abspath("../backend"))

from utils.api_data_fetcher import API_Fetcher
import asyncio
import yfinance as yf
import pandas as pd
from statsforecast import StatsForecast
from statsforecast.models import ARIMA
from datetime import datetime
from neuralforecast import NeuralForecast
from neuralforecast.models import RNN, NHITS
import ipywidgets as widgets
from IPython.display import display


Warning - Certain functionality 
             requires requests_html, which is not installed.
             
             Install using: 
             pip install requests_html
             
             After installation, you may have to restart your Python session.


## Async Context: fetch all important tickers

In [2]:
fetcher = API_Fetcher()
tickers = await fetcher.fetch_all_major_indices()

## Dropdown-Widget

Please select one of the tickers in the drop down list.

In [3]:
ticker_dropdown = widgets.Dropdown(
    options=tickers,
    description='Ticker:',
    value='AAPL',
    style={'description_width': 'initial'}
)

display(ticker_dropdown)


Dropdown(description='Ticker:', options=('AAPL', 'AMGN', 'AMZN', 'AXP', 'BA', 'CAT', 'CRM', 'CSCO', 'CVX', 'DI…

## Get historical data for selected ticker

In [20]:
selected_ticker = ticker_dropdown.value
data = await fetcher.fetch_selected_stock_data_yf(selected_ticker)


## Historical price trends


In [21]:
data['price_history'].head()

Date
2015-06-01 00:00:00-04:00    21.704500
2015-07-01 00:00:00-04:00    26.807501
2015-08-01 00:00:00-04:00    25.644501
2015-09-01 00:00:00-04:00    25.594500
2015-10-01 00:00:00-04:00    31.295000
Name: Close, dtype: float64


# Data preparation

In [ ]:
df = data['price_history'].reset_index() 
df.columns = ['ds', 'y']  # Rename columns to match Nixtla's required format, 'ds' = datetime and 'y' = value
df['unique_id'] = selected_ticker  # Important for multi-series models in Nixtla
 
# Add features to the DataFrame
df['recommendation'] = 1 if data['recommendation_key'] == 'buy' else 0 # Binary Feature
df['eps_forward'] = data['eps_forward']
df['revenue_growth'] = data['revenue_growth']
df['recommendation_mean'] = data['recommendation_mean']
df['gross_margins'] = data['gross_margins']
df['dividend_yield'] = data['dividend_yield']
df['debt_to_equity'] = data['debt_to_equity']

feature_columns = [
    'recommendation', 'eps_forward', 'revenue_growth', 
    'recommendation_mean', 'gross_margins', 
    'dividend_yield', 'debt_to_equity'
]

# Fill NaN values in feature columns with 0.0 
for col in feature_columns:
    if col in df.columns:
        df[col] = df[col].fillna(0.0) 

df.tail()


C:\Users\User\AppData\Local\Temp\ipykernel_6224\1846170522.py:23: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].fillna(0.0)  # Oder ein anderer Wert wie df[col].mean()


,ds,y,unique_id,recommendation,eps_forward,revenue_growth,recommendation_mean,gross_margins,dividend_yield,debt_to_equity
115,2025-01-01 00:00:00-05:00,237.679993,AMZN,0,6.15,0.086,1.40845,0.49158,0.0,43.563
116,2025-02-01 00:00:00-05:00,212.279999,AMZN,0,6.15,0.086,1.40845,0.49158,0.0,43.563
117,2025-03-01 00:00:00-05:00,190.259995,AMZN,0,6.15,0.086,1.40845,0.49158,0.0,43.563
118,2025-04-01 00:00:00-04:00,184.419998,AMZN,0,6.15,0.086,1.40845,0.49158,0.0,43.563
119,2025-05-01 00:00:00-04:00,189.979996,AMZN,0,6.15,0.086,1.40845,0.49158,0.0,43.563


# Model training & forecast

In [25]:
# Create a recurrent neural network for forecasting
model = RNN(h=5, input_size=12, max_steps=100)

nf = NeuralForecast(
    models=[model],
    freq='M'  # Monthly data
)

# Fit model on prepared dataset
nf.fit(df=df)

# Make prediction
forecast = nf.predict()
forecast.head()

Seed set to 1
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name         | Type          | Params | Mode 
-------------------------------------------------------
0 | loss         | MAE           | 0      | train
1 | padder_train | ConstantPad1d | 0      | train
2 | scaler       | TemporalNorm  | 0      | train
3 | hist_encoder | RNN           | 49.8 K | train
4 | mlp_decoder  | MLP           | 16.6 K | train
-------------------------------------------------------
66.4 K    Trainable params
0         Non-trainable params
66.4 K    Total params
0.266     Total estimated model params size (MB)
10        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=100` reached.
c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\utilsforecast\processing.py:384: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  freq = pd.tseries.frequencies.to_offset(freq)
c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\utilsforecast\processing.py:440: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  freq = pd.tseries.frequencies.to_offset(freq)
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

,unique_id,ds,RNN
0,AMZN,2025-05-31 00:00:00-04:00,225.754547
1,AMZN,2025-06-30 00:00:00-04:00,214.803375
2,AMZN,2025-07-31 00:00:00-04:00,199.448639
3,AMZN,2025-08-31 00:00:00-04:00,200.520752
4,AMZN,2025-09-30 00:00:00-04:00,205.361694


# Forecast analysis

In [26]:
# Last known real price
current_price = df['y'].iloc[-1]

# Average of the forecasted prices
mean_forecast_price = forecast['RNN'].mean()

# Calculate absolute and percentage difference
price_diff = mean_forecast_price - current_price
percent_change = (price_diff / current_price) * 100

print(f"Current: {current_price:.2f}, Forecast: {mean_forecast_price:.2f}, Change: {percent_change:.2f}%")

Current: 189.98, Forecast: 209.18, Change: 10.11%


# Trading strategy

In [27]:
def trading_strategy(percent_change):
    if percent_change > 5:
        return f"📈 BUY CALL: Expected increase of {percent_change:.2f}% - use possible opportunity!"
    elif percent_change < -5:
        return f"📉 BUY PUT: Expected decrease of {abs(percent_change):. 2f}% - hedge risk!"
    elif -2 <= percent_change <= 2:
        return f"➖ HOLD / SELL COVERED CALL: Movement below 2% - sideways market."
    else:
        return f"🔁 STRADDLE / SPREAD: Movement possible, direction unclear (±{percent_change:.2f}%)"

print(trading_strategy(percent_change))

📈 BUY CALL: Expected increase of 10.11% - use possible opportunity!
